In [38]:
"""
This implements the velocity-controlled point pusher system. We simplify the analysis by assuming that
the relative velocity is to be passed on the slider from the pusher, under the assumption that at small
velocity the acceleration acts in the direction of the realtive velocity until objects move. The effect
of the force and acceleration are to be ignored, hence a quasi-static system is assumed.

Different modes are to be considered:
1) simple model: ignore friction between pusher and slider. Only pass the normal velocity to the slider.
This creates a translating velocity and a rotation velocity
2) friction model: assume the friction cone to be certain shapes, and consider the sticking and sliding modes.

Object models to be considered:
1) cylinders
2) blocks
"""
import numpy as np
class Slider:
    def __init__(self, transform, shape):
        # transform: x,y,theta
        # shape: cylinder is radius, block is (height, width)
        self.transform = transform
        x,y,theta = transform
        tf_mat = [[np.cos(theta), -np.sin(theta),x], [np.sin(theta), np.cos(theta),y],[0,0,1]]
        tf_mat = np.array(tf_mat)
        self.tf_mat = tf_mat
        self.shape = shape
        self.tran_vel = np.zeros((2))
        self.rot_vel = 0
        self.type = 'none'
    def get_tf_mat(self, transform):
        x,y,theta = transform
        tf_mat = [[np.cos(theta), -np.sin(theta),x], [np.sin(theta), np.cos(theta),y],[0,0,1]]
        tf_mat = np.array(tf_mat)
        return tf_mat
    def update_transform(self, new_transform):
        self.transform = new_transform
        self.tf_mat = self.get_tf_mat(new_transform)
    def update_vel(self, tran_vel, rot_vel):
        self.tran_vel = tran_vel
        self.rot_vel = rot_vel
    def get_contact_pt(self, contact_angle):
        pass
    def get_normal_local(self, contact_angle):
        # any convex object can map 1-to-1 to a unit circle
        # this is in the local frame
        pass
    def get_tran_vel(self, contact_angle, vel):
        # get the vector from contact point to COM
        pt = self.get_contact_pt(contact_angle)
        pt = pt/np.linalg.norm(pt)
        return vel.dot(pt)*pt
    def get_rot_vel(self, contact_angle, vel):
        # get the vector from contact point to COM
        pt = self.get_contact_pt(contact_angle)
        norm = np.linalg.norm(pt)
        pt = pt/np.linalg.norm(pt)
        # linear_v = vel - vel.dot(norm)*norm
        linear_v = np.cross(vel, pt)  # |a||b|sin(theta)
        ang_v = linear_v / norm
        return ang_v
    def get_vel_at_contact(self, contact_angle, tran_vel, rot_vel):
        # get the contact poitn velocity, assuming the object is undergoing velocity
        pass
    def get_cur_vel_at_contact(self, contact_angle):
        # get the contact point velocity based on the object current velocity
        pass


# TODO: implement cylinder and block sliders. Write an animation for these
class CylinderSlider(Slider):
    def __init__(self, transform, shape):
        super().__init__(transform, shape)
        self.radius = shape  # shape is a float
        self.type = 'cylinder'
    def get_contact_pt(self, contact_angle):
        return self.radius*np.array([np.cos(contact_angle),np.sin(contact_angle)])
    def get_contact_pt_in_world(self, contact_angle):
        contact_pt = self.get_contact_pt(contact_angle)
        return self.tf_mat[:2,:2].dot(contact_pt) + self.tf_mat[:2,2]
    def get_normal_local(self, contact_angle):
        return -np.array([np.cos(contact_angle),np.sin(contact_angle)])
    def get_vel_at_contact(self, contact_angle, tran_vel, rot_vel):
        linear_v = rot_vel * self.radius
        linear_v = np.array([0,0,linear_v])
        # get the vector pointing from the COM to the contact point. This is useful to get the
        # linear velocity at the contact point
        contact_pt = self.get_contact_pt(contact_angle)
        contact_pt_vec = self.tf_mat[:2,:2].dot(contact_pt)
        contact_pt_vec = contact_pt_vec / np.linalg.norm(contact_pt_vec)
        contact_pt_vec = np.array([contact_pt_vec[0], contact_pt_vec[1], 0])
        linear_v = np.cross(linear_v,contact_pt_vec)[:2]
        return tran_vel + linear_v
    def get_cur_vel_at_contact(self, contact_angle):
        # get the contact point velocity based on the object current velocity
        return self.get_vel_at_contact(contact_angle, self.tran_vel, self.rot_vel)

    
class BlockSlider(Slider):
    def __init__(self, transform, shape):
        super().__init__(transform, shape)
        self.width = shape[0]
        self.height = shape[1]
        self.shape = np.array(shape[:2])
        self.type = 'block'
    def get_contact_pt(self, contact_angle):
        pt = np.array([np.cos(contact_angle),np.sin(contact_angle)])
        # map to rectangular
        ratio = np.array([self.width,self.height])/2/np.abs(pt)
        ratio = np.min(ratio)
        pt = pt * ratio
        return pt
    def get_contact_pt_in_world(self, contact_angle):
        contact_pt = self.get_contact_pt(contact_angle)
        return self.tf_mat[:2,:2].dot(contact_pt) + self.tf_mat[:2,2]
    def get_normal_local(self, contact_angle):
        # this is useful to project the velocity to ignore friction
        pt = self.get_contact_pt(contact_angle)
        if (pt[0] > -self.width/2) and (pt[0] < self.width/2):
            norm = -np.array([0,np.sign(pt[1])])
        if (pt[1] > -self.height/2) and (pt[1] < self.height/2):
            norm = -np.array([np.sign(pt[0]),0])
        return norm
    def get_vel_at_contact(self, contact_angle, tran_vel, rot_vel):
        pt = self.get_contact_pt(contact_angle)
        linear_v = rot_vel * np.linalg.norm(pt)
        linear_v = np.array([0,0,linear_v])
        # get the vector pointing from the COM to the contact point. This is useful to get the
        # linear velocity at the contact point
        contact_pt = self.get_contact_pt(contact_angle)
        contact_pt_vec = self.tf_mat[:2,:2].dot(contact_pt)
        contact_pt_vec = contact_pt_vec / np.linalg.norm(contact_pt_vec)
        contact_pt_vec = np.array([contact_pt_vec[0], contact_pt_vec[1], 0])
        linear_v = np.cross(linear_v,contact_pt_vec)[:2]
        return tran_vel + linear_v
    def get_cur_vel_at_contact(self, contact_angle):
        # get the contact point velocity based on the object current velocity
        return self.get_vel_at_contact(contact_angle, self.tran_vel, self.rot_vel)

    
    

# TODO: what about robot models? How to infer that?
# maybe we can randomly select a point on the robot arm geometry, and assuming convex shape. Then
# iteratively refine until the robot geometry roughly covers the point and is normal to that.
# This pushing motion will be noisy, so we need a controller to decide the velocity after a certain
# period for feedback control

class System:
    def __init__(self, slider: Slider, dt):
        self.slider = slider
        self.dt = dt
    def get_contact_point(self, pusher_pt):
        """
        at current slider transform, get the current contact point given the pusher point in the world frame
        """
        pass
    def update_slider_transform(self, s0, v0, v1):
        """
        v0: velocity before updating the velocity of the slider. (e.g. default is static)
        v1: velocity after update
        both v0 and v1 are of form [vx, vy, omega]
        """
        # we use a simple euler integration
        s1 = s0 + self.dt * v1
        return s1

    def update_slider_rel(self, contact_angle, pusher_rel_vel):
        """
        given the contact angle, update the slider velocity and transform.
        The pusher_rel_velocity is in the frame of the slider
        NOTE: in quasi-static assumption, we basically assume that the contact velocity is equivalent
        to the pusher velocity in the slider after the pusher-slider friction effect
        """
        # frictionless: we ignore the tangent velocity to the boundary, and only take the normal velocity
        norm = self.slider.get_normal_local(contact_angle)
#         print('norm: ', norm)
        pusher_rel_vel = pusher_rel_vel.dot(norm) * norm

#         print("pusher_rel_vel after projecting: ", pusher_rel_vel)
        rel_tran_vel = self.slider.get_tran_vel(contact_angle, pusher_rel_vel)
        rel_rot_vel = self.slider.get_rot_vel(contact_angle, pusher_rel_vel)
        # TODO: map the relative velocity to world frame, and update
        tran_vel = self.slider.tran_vel + self.slider.tf_mat[:2,:2].dot(rel_tran_vel)
        rot_vel = self.slider.rot_vel + rel_rot_vel
        v0 = np.array([self.slider.tran_vel[0],self.slider.tran_vel[1],self.slider.rot_vel])
        v1 = np.array([tran_vel[0],tran_vel[1],rot_vel])
        transform = self.update_slider_transform(self.slider.transform, v0, v1)
        self.slider.update_transform(transform)
        self.slider.update_vel(tran_vel, rot_vel)

    def update_slider(self, contact_angle, pusher_vel_in_slider):
        """
        given the contact angle, update the slider velocity and transform.
        The pusher_vel_in_slider is in the frame of the slider
        NOTE: in quasi-static assumption, we basically assume that the contact velocity is equivalent
        to the pusher velocity in the slider after the pusher-slider friction effect
        """
        # compute the relative velocity of the pusher w.r.t. the slider
        contact_vel = self.slider.get_cur_vel_at_contact(contact_angle) # in world frame
        contact_vel_in_slider = self.slider.tf_mat[:2,:2].dot(contact_vel)
#         print('contact_vel_in_slider: ', contact_vel_in_slider)
        pusher_rel_vel = pusher_vel_in_slider - contact_vel_in_slider
#         print('pusher_rel_vel: ', pusher_rel_vel)
        self.update_slider_rel(contact_angle, pusher_rel_vel)
        
        print('slider translation velocity: ', self.slider.tran_vel)
        print('slider rotation velocity: ', self.slider.rot_vel)


    


In [39]:
class PointPusher:
    def __init__(self, start_position, waypoints, velocity):
        self.start_position = start_position
        self.waypoints = waypoints
        self.next_waypoint_i = 1
        self.velocity = velocity


In [40]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import matplotlib as mpl
from IPython import display
"""
implement the animation for the pusher-slider system
"""
class SystemAnimationRelativePusher:
    def __init__(self, slider: Slider, pusher_contact_angle, pusher_rel_vel, dt):
        self.system = System(slider, dt)
        self.pusher_contact_angle = pusher_contact_angle
        self.pusher_rel_vel = pusher_rel_vel
        self.fig, self.ax = plt.subplots(figsize=(8, 8))
    
    def get_ori(self):
        # get the orientation x0, x1, y0, y1 for the slider
        vec = self.system.slider.tf_mat[:2,:2].dot([1,0]) * 0.1
        start_pos = self.system.slider.transform[:2]
        end_pos = start_pos + vec
        return start_pos, end_pos

    def init_anim(self):
        # plot the slider at the start position
        if self.system.slider.type == 'cylinder':
            self.slider_anim = plt.Circle((self.system.slider.transform[0],self.system.slider.transform[1]),
                                self.system.slider.radius, color='b')
            start_ori, end_ori = self.get_ori()
            self.slider_ori_anim, = self.ax.plot([start_ori[0], end_ori[0]],
                                           [start_ori[1], end_ori[1]],
                                           color='r')
        else:
            self.slider_anim = plt.Rectangle(self.system.slider.transform[:2]-self.system.slider.shape/2,
                                             self.system.slider.width,
                                             self.system.slider.height,
                                             color='b')
            transform = self.system.slider.transform
            t = mpl.transforms.Affine2D().rotate_deg_around(transform[0], transform[1], \
                                                            transform[2]/np.pi * 180) + self.ax.transData
            self.slider_anim.set_transform(t)
            start_ori, end_ori = self.get_ori()
            self.slider_ori_anim, = self.ax.plot([start_ori[0], end_ori[0]],
                                           [start_ori[1], end_ori[1]],
                                           color='r')
            
        # plot the pusher
        # get the transform of the pusher in the world frame
        pusher_pt = self.system.slider.get_contact_pt_in_world(self.pusher_contact_angle)
        self.pusher_anim = plt.Circle((pusher_pt[0], pusher_pt[1]), 0.01, color='green')
        self.ax.set_xlim(-2, 2)
        self.ax.set_ylim(-2, 2)

        self.ax.add_patch(self.slider_anim)
        self.ax.add_patch(self.pusher_anim)
        return self.slider_anim, self.pusher_anim, self.slider_ori_anim, 
    def step(self, i):
        print('step: ', i)
        # update the slider based on the pusher
        self.system.update_slider(self.pusher_contact_angle, self.pusher_rel_vel)
        if self.system.slider.type == 'cylinder':
            self.slider_anim.center = self.system.slider.transform[0],self.system.slider.transform[1]
        else:
            transform = self.system.slider.transform
            self.slider_anim.set_xy([transform[0]-self.system.slider.width/2, 
                                     transform[1]-self.system.slider.height/2])
            t = mpl.transforms.Affine2D().rotate_deg_around(transform[0], transform[1], \
                                                            transform[2]/np.pi * 180) + self.ax.transData
            self.slider_anim.set_transform(t)      
        start_ori, end_ori = self.get_ori()
#         print('start ori: ', start_ori)
#         print('start ori: ', start_ori)

        self.slider_ori_anim.set_xdata([start_ori[0], end_ori[0]])
        self.slider_ori_anim.set_ydata([start_ori[1], end_ori[1]])

        # get the relative position again
        pusher_pt = self.system.slider.get_contact_pt_in_world(self.pusher_contact_angle)
        self.pusher_anim.center = pusher_pt[0],pusher_pt[1]
        
        return self.slider_anim, self.pusher_anim,
    def anim(self):
        ani = FuncAnimation(self.fig, self.step, frames=np.arange(0,400,1),
                            init_func=self.init_anim, interval=int(1000*dt), blit=False)
        video = ani.to_html5_video()
        html = display.HTML(video)
        display.display(html)
        plt.close()


In [41]:
# slider = CylinderSlider([0,0,0], 0.2)
slider = BlockSlider([0,0,0], [0.2,0.2])

pusher_contact_angle = 3/2*np.pi + 30*np.pi/180
pusher_rel_vel = [0,0.1]
dt = 0.01
anim = SystemAnimationRelativePusher(slider, pusher_contact_angle, pusher_rel_vel, dt)
anim.anim()

step:  0
slider translation velocity:  [-0.04330127  0.075     ]
slider rotation velocity:  -0.43301270189221935
step:  1
slider translation velocity:  [-0.06454748  0.1121703 ]
slider rotation velocity:  -0.6470823643718557
step:  2
slider translation velocity:  [-0.07435896  0.12959627]
slider rotation velocity:  -0.7470735327044274
step:  3
slider translation velocity:  [-0.07831691  0.13675041]
slider rotation velocity:  -0.7879535744985041
step:  4
slider translation velocity:  [-0.07937464  0.13869836]
slider rotation velocity:  -0.79903654937354
step:  5
slider translation velocity:  [-0.07906767  0.1381221 ]
slider rotation velocity:  -0.7957719631436937
step:  6
slider translation velocity:  [-0.07816983  0.1364038 ]
slider rotation velocity:  -0.7860783254181696
step:  7
slider translation velocity:  [-0.07705552  0.13422976]
slider rotation velocity:  -0.7738634024292207
step:  8
slider translation velocity:  [-0.07589678  0.13192528]
slider rotation velocity:  -0.7609664144

step:  78
slider translation velocity:  [-0.0547901   0.06676829]
slider rotation velocity:  -0.4165819595113696
step:  79
slider translation velocity:  [-0.05475571  0.06636037]
slider rotation velocity:  -0.41453510198395244
step:  80
slider translation velocity:  [-0.05472349  0.06595827]
slider rotation velocity:  -0.41251815538130654
step:  81
slider translation velocity:  [-0.05469337  0.06556186]
slider rotation velocity:  -0.4105304159122935
step:  82
slider translation velocity:  [-0.05466529  0.06517103]
slider rotation velocity:  -0.4085712040245743
step:  83
slider translation velocity:  [-0.05463918  0.06478564]
slider rotation velocity:  -0.40663986333716334
step:  84
slider translation velocity:  [-0.05461498  0.06440559]
slider rotation velocity:  -0.4047357596307894
step:  85
slider translation velocity:  [-0.05459264  0.06403076]
slider rotation velocity:  -0.40285827989237216
step:  86
slider translation velocity:  [-0.0545721   0.06366104]
slider rotation velocity: 

step:  158
slider translation velocity:  [-0.05564725  0.04605238]
slider rotation velocity:  -0.31256852005876284
step:  159
slider translation velocity:  [-0.05567938  0.04589444]
slider rotation velocity:  -0.3117626941039773
step:  160
slider translation velocity:  [-0.05571166  0.04573823]
slider rotation velocity:  -0.31096510341770706
step:  161
slider translation velocity:  [-0.05574409  0.04558371]
slider rotation velocity:  -0.31017566876759156
step:  162
slider translation velocity:  [-0.05577667  0.04543087]
slider rotation velocity:  -0.3093943129387179
step:  163
slider translation velocity:  [-0.05580938  0.0452797 ]
slider rotation velocity:  -0.3086209606975171
step:  164
slider translation velocity:  [-0.05584221  0.04513018]
slider rotation velocity:  -0.30785553875708704
step:  165
slider translation velocity:  [-0.05587516  0.04498229]
slider rotation velocity:  -0.307097975743902
step:  166
slider translation velocity:  [-0.05590823  0.04483603]
slider rotation ve

step:  238
slider translation velocity:  [-0.05812266  0.03798614]
slider rotation velocity:  -0.27029428545159345
step:  239
slider translation velocity:  [-0.05814443  0.03793876]
slider rotation velocity:  -0.27003355810753754
step:  240
slider translation velocity:  [-0.05816574  0.03789273]
slider rotation velocity:  -0.2697799582260509
step:  241
slider translation velocity:  [-0.05818656  0.03784806]
slider rotation velocity:  -0.269533549108382
step:  242
slider translation velocity:  [-0.05820688  0.03780476]
slider rotation velocity:  -0.26929439710633285
step:  243
slider translation velocity:  [-0.05822669  0.03776285]
slider rotation velocity:  -0.2690625717199786
step:  244
slider translation velocity:  [-0.05824598  0.03772232]
slider rotation velocity:  -0.2688381457003498
step:  245
slider translation velocity:  [-0.05826474  0.03768319]
slider rotation velocity:  -0.2686211951573412
step:  246
slider translation velocity:  [-0.05828294  0.03764547]
slider rotation vel

step:  318
slider translation velocity:  [-0.05601192  0.04080143]
slider rotation velocity:  -0.287836685132376
step:  319
slider translation velocity:  [-0.0558592   0.04100352]
slider rotation velocity:  -0.2891032098372657
step:  320
slider translation velocity:  [-0.05569832  0.04121514]
slider rotation velocity:  -0.29043233193558043
step:  321
slider translation velocity:  [-0.05552879  0.04143679]
slider rotation velocity:  -0.2918276161305102
step:  322
slider translation velocity:  [-0.05535008  0.04166905]
slider rotation velocity:  -0.29329291421470316
step:  323
slider translation velocity:  [-0.0551616   0.04191252]
slider rotation velocity:  -0.29483239427270147
step:  324
slider translation velocity:  [-0.05496273  0.04216784]
slider rotation velocity:  -0.29645057348180937
step:  325
slider translation velocity:  [-0.05475279  0.04243574]
slider rotation velocity:  -0.29815235503238807
step:  326
slider translation velocity:  [-0.05453104  0.04271698]
slider rotation v

step:  398
slider translation velocity:  [34.58240527 13.99756852]
slider rotation velocity:  -0.00015350578916661223
step:  399
slider translation velocity:  [34.58239899 13.99761431]
slider rotation velocity:  7.759240079948602e-05
